In [1]:
import os
import pandas as pd

from dotenv import load_dotenv
from pathlib import Path

# from dataprep.eda import create_report, diff

%load_ext autoreload
%autoreload 2

In [2]:
pd.options.display.float_format = "{:.3f}".format

In [3]:
load_dotenv(override=True)

True

In [4]:
DIR_DATA_RAW: str = Path(os.getenv("DIR_DATA_RAW"))
DIR_OUTPUTS: str = Path(os.getenv("DIR_OUTPUTS"))

In [5]:
df = pd.read_csv(f"{DIR_DATA_RAW}/PS_20174392719_1491204439457_log.csv")
df = df.rename(
    columns={
        "oldbalanceOrg": "oldBalanceOrig",
        "newbalanceOrig": "newBalanceOrig",
        "oldbalanceDest": "oldBalanceDest",
        "newbalanceDest": "newBalanceDest",
    }
)

In [6]:
df.describe()

,step,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
count,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000
mean,243.397,179861.904,833883.104,855113.669,1100701.667,1224996.398,0.001,0.000
std,142.332,603858.231,2888242.673,2924048.503,3399180.113,3674128.942,0.036,0.002
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,156.000,13389.570,0.000,0.000,0.000,0.000,0.000,0.000
50%,239.000,74871.940,14208.000,0.000,132705.665,214661.440,0.000,0.000
75%,335.000,208721.478,107315.175,144258.410,943036.708,1111909.250,0.000,0.000
max,743.000,92445516.640,59585040.370,49585040.370,356015889.350,356179278.920,1.000,1.000


In [7]:
df.isnull().values.any()

False

In [8]:
print(
    f"The types of fraudulent transactions are {df.loc[df['isFraud'] == 1, 'type'].unique()}"
)

fraud_transfer_df = df.loc[(df["isFraud"] == 1) & (df["type"] == "TRANSFER")]
fraud_cashout_df = df.loc[(df["isFraud"] == 1) & (df["type"] == "CASH_OUT")]

print(f"The number of fraudulent TRANSFERs = {fraud_transfer_df.shape[0]}")

print(f"The number of fraudulent CASH_OUTs = {fraud_cashout_df.shape[0]}")

The types of fraudulent transactions are ['TRANSFER' 'CASH_OUT']
The number of fraudulent TRANSFERs = 4097
The number of fraudulent CASH_OUTs = 4116


Only 2 of the 5 types of transactions are concerned by fraud cases ('TRANSFER' & 'CASH_OUT') almost same number.

In [9]:
print(
    f"The type of transactions in which isFlaggedFraud is set: {df.loc[df['isFlaggedFraud'] == 1, 'type'].unique()}"
)

transfer_df = df.loc[df["type"] == "TRANSFER"]
flagged_df = df.loc[df["isFlaggedFraud"] == 1]
not_flagged_df = df.loc[df["isFlaggedFraud"] == 0]

print(f"Min amount transacted when isFlaggedFraud is set= {flagged_df['amount'].min()}")
print(
    f"Max amount transacted in a TRANSFER where isFlaggedFraud is not set= {transfer_df.loc[transfer_df['isFlaggedFraud'] == 0, 'amount'].max()}"
)

The type of transactions in which isFlaggedFraud is set: ['TRANSFER']
Min amount transacted when isFlaggedFraud is set= 353874.22
Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64


In [10]:
print(
    f"The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: {transfer_df.loc[(transfer_df['isFlaggedFraud'] == 0) & (transfer_df['oldBalanceDest'] == 0) & (transfer_df['newBalanceDest'] == 0)].shape[0]}"
)

The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: 4158


In [11]:
print(
    f"Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {[flagged_df['oldBalanceOrig'].min(), flagged_df['oldBalanceOrig'].max()]}"
)

iso_balance = transfer_df.loc[
    (transfer_df["isFlaggedFraud"] == 0)
    & (transfer_df["oldBalanceOrig"] == transfer_df["newBalanceOrig"]),
    "oldBalanceOrig",
]

print(
    f"Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: {iso_balance.min(), iso_balance.max()}"
)

Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: [353874.22, 19585040.37]
Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: (0.0, 575667.54)


In [12]:
print(
    f"Have originators of transactions flagged as fraud transacted more than once? {not_flagged_df['nameOrig'].isin(pd.concat([not_flagged_df['nameOrig'], not_flagged_df['nameDest']])).any()}"
)

print(
    f"Have destinations for transactions flagged as fraud initiated other transactions? {flagged_df['nameDest'].isin(not_flagged_df['nameOrig']).any()}"
)

print(
    f"How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: {sum(flagged_df['nameDest'].isin(not_flagged_df['nameDest']))}"
)

Have originators of transactions flagged as fraud transacted more than once? True
Have destinations for transactions flagged as fraud initiated other transactions? False
How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


In [13]:
print(
    f"Are there any merchants among originator accounts for CASH_IN transactions? {df.loc[df['type'] == 'CASH_IN', 'nameOrig'].str.contains('M').any()}"
)

Are there any merchants among originator accounts for CASH_IN transactions? False


In [14]:
print(
    f"Are there any merchants among originator accounts for CASH_IN transactions? {df.loc[df['type'] == 'CASH_OUT', 'nameOrig'].str.contains('M').any()}"
)

Are there any merchants among originator accounts for CASH_IN transactions? False


In [15]:
print(
    f"Are there merchants among any originator accounts? {df['nameOrig'].str.contains('M').any()}"
)

print(
    f"Are there any transactions having merchants among destination accounts other than the PAYMENT type? {(df.loc[df['nameDest'].str.contains('M'), 'type'] != 'PAYMENT').any()}"
)

Are there merchants among any originator accounts? False
Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [16]:
print(
    f"Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? {(fraud_transfer_df['nameDest'].isin(fraud_cashout_df['nameOrig'])).any()}"
)

not_fraud_df = df.loc[df["isFraud"] == 0]
fraud_df = df.loc[df["isFraud"] == 1]

Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [17]:
print(
    f"Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: \n\n {fraud_transfer_df.loc[fraud_transfer_df['nameDest'].isin(not_fraud_df.loc[not_fraud_df['type'] == 'CASH_OUT', 'nameOrig'])].drop_duplicates()}"
)

Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

          step      type      amount     nameOrig  oldBalanceOrig  \
1030443    65  TRANSFER 1282971.570  C1175896731     1282971.570   
6039814   486  TRANSFER  214793.320  C2140495649      214793.320   
6362556   738  TRANSFER  814689.880  C2029041842      814689.880   

         newBalanceOrig     nameDest  oldBalanceDest  newBalanceDest  isFraud  \
1030443           0.000  C1714931087           0.000           0.000        1   
6039814           0.000   C423543548           0.000           0.000        1   
6362556           0.000  C1023330867           0.000           0.000        1   

         isFlaggedFraud  
1030443               0  
6039814               0  
6362556               0  


In [18]:
print(
    f"Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = {not_fraud_df.loc[(not_fraud_df['type'] == 'CASH_OUT') & (not_fraud_df['nameOrig'] == 'C423543548'), 'step'].values}"
)

Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = [185]


In [19]:
print(
    f"The fraction of fraudulent transactions with 'oldBalanceDest'='newBalanceDest' = 0 although the transacted 'amount' is non-zero is: {fraud_df.loc[(fraud_df['oldBalanceDest'] == 0) & (fraud_df['newBalanceDest'] == 0) & (fraud_df['amount'] != 0)].shape[0]/fraud_df.shape[0]}"
)

print(
    f"The fraction of genuine transactions with 'oldBalanceDest'='newBalanceDest' = 0 although the transacted 'amount' is non-zero is: {not_fraud_df.loc[(not_fraud_df['type'].isin(['TRANSFER', 'CASH_OUT'])) & (not_fraud_df['oldBalanceDest'] == 0) & (not_fraud_df['newBalanceDest'] == 0) & (not_fraud_df['amount'] != 0)].shape[0]/not_fraud_df[(not_fraud_df['type'].isin(['TRANSFER', 'CASH_OUT']))].shape[0]}"
)

The fraction of fraudulent transactions with 'oldBalanceDest'='newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4955558261293072
The fraction of genuine transactions with 'oldBalanceDest'='newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0006176245277308345
